In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import pandas as pd

pd.set_option('display.max_rows', None)

# Getting Player Data from Basketball Reference

In [2]:
BASE_URL = "https://www.basketball-reference.com/players/"
LETTERS = "abcdefghijklmnopqrstuvwxyz"

In [3]:
# CSV File Setup
with open("basketball_players.csv", "w", newline="", encoding="utf-8-sig") as file:
    writer = csv.writer(file)
    writer.writerow(["Name", "From", "To", "Pos", "Ht", "Wt", "Birth Date", "Colleges", "CSV_ID"])
    for letter in LETTERS:
        url = f"{BASE_URL}{letter}/"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve {url}")
            continue
            
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table", {"id": "players"})
        
        if not table:
            print(f"No player table found for {letter}")
            continue
        
        for row in table.find("tbody").find_all("tr", class_=lambda x: x != "thead"):
            cols = row.find_all("td")
            if not cols:
                continue
            
            name = row.find("th").text.strip()
            csv_id = row.find("th")["data-append-csv"]
            from_year = cols[0].text.strip()
            to_year = cols[1].text.strip()
            pos = cols[2].text.strip() if len(cols) > 2 else ""
            height = cols[3].text.strip() if len(cols) > 3 else ""
            weight = cols[4].text.strip() if len(cols) > 4 else ""
            birth_date = cols[5].text.strip() if len(cols) > 5 else ""
            colleges = cols[6].text.strip() if len(cols) > 6 else ""
            
            writer.writerow([name, from_year, to_year, pos, height, weight, birth_date, colleges, csv_id])
        
        print(f"Finished scraping letter: {letter}")
        time.sleep(5)  # Pause to prevent being blocked

Finished scraping letter: a
Finished scraping letter: b
Finished scraping letter: c
Finished scraping letter: d
Finished scraping letter: e
Finished scraping letter: f
Finished scraping letter: g
Finished scraping letter: h
Finished scraping letter: i
Finished scraping letter: j
Finished scraping letter: k
Finished scraping letter: l
Finished scraping letter: m
Finished scraping letter: n
Finished scraping letter: o
Finished scraping letter: p
Finished scraping letter: q
Finished scraping letter: r
Finished scraping letter: s
Finished scraping letter: t
Finished scraping letter: u
Finished scraping letter: v
Finished scraping letter: w
Finished scraping letter: x
Finished scraping letter: y
Finished scraping letter: z


In [4]:
players = pd.read_csv("basketball_players.csv", encoding="utf-8-sig")

players[['fullname', 'hall_of_fame']] = players['Name'].str.split('*', expand = True)

players['hall_of_fame'] = players['hall_of_fame'].str.replace("", "y")

players['hall_of_fame'] = players['hall_of_fame'].fillna('n')

players['hall_of_fame'] = players['hall_of_fame'].str.replace('None', 'n')

players['Wt'] = players['Wt'].fillna(0).astype(int)

players['last_college'] = players['Colleges'].str.split(',').str[-1].str.strip()

players['birth_date'] = pd.to_datetime(players['Birth Date'], format = '%B %d, %Y')

players = players.rename(columns = {'Ht': 'height',
                                    'Wt': 'weight',
                                    'From': 'from_year',
                                    'To': 'to_year',
                                    'Pos': 'position',
                                    'CSV_ID':'player_id',})

players = players[['player_id', 'fullname', 'height', 'weight', 'position', 'from_year', 'to_year', 'birth_date', 'hall_of_fame']]

In [6]:
players.to_csv('player_data_cleaned.csv', encoding = 'utf-8-sig', index = False)

# Reading in Schedule Data and Team data to get schedule data into the database format

## Changes that need made to schedule data

1) Change date to yyyy-mm-dd format

2) Start time needs to be put into military time

4) Arena needs to be replaced with neutral site indicator. I will see if the venue does not match the home team's assigned venue and if it does not, the neutral site indicator will become a yes

5) I may add in nba cup game indicator

6) Change overtime indicator to yes or no instead of OT

In [134]:
schedule = pd.read_csv("schedule.csv", dtype = "str")

teams = pd.read_csv("teams.csv", dtype = "str")

In [ ]:
schedule['date'] = pd.to_datetime(schedule['date'], format = '%m/%d/%Y')

# Preprocess the time column to ensure two digits for the hour
schedule['Start (ET)'] = schedule['Start (ET)'].apply(lambda x: x if len(x.split(':')[0]) == 2 else '0' + x)

schedule['Start (ET)'] = schedule['Start (ET)'].str.replace('p', 'PM').replace('a', 'AM')

schedule['start_time'] = pd.to_datetime(schedule['Start (ET)'], format='%I:%M%p').dt.strftime('%H:%M')

schedule['overtime'] = schedule['overtime'].fillna('0')

schedule['overtime'] = schedule['overtime'].str.replace('OT', '1').str[0]

schedule['Arena'] = schedule['Arena'].str.replace('Madison Square Garden (IV)', 'Madison Square Garden')

merged = schedule.merge(teams, left_on = ['home', 'Arena'], right_on = ['fullname', 'home_venue'], how = 'left', indicator = True)

# Add a new column to df1 that marks 'y' if the pair does not exist in df2
schedule['neutral_site'] = merged['_merge'].apply(lambda x: 'y' if x == 'left_only' else '')

schedule['neutral_site'] = schedule['neutral_site'].str.strip().replace('', 'n')

schedule['game_length'] = schedule['game_length'].str.zfill(5) + ':00'

schedule = schedule[['date', 'visitor', 'home', 'start_time', 'neutral_site', 'overtime', 'home_points', 'away_points', 'game_length']]

In [144]:
schedule.to_csv("schedule_cleaned.csv", index = False)

In [ ]:
teams['bbreference_abbreviation'] = teams['team_id']

teams['bbreference_abbreviation'] = teams['bbreference_abbreviation'].str.replace('CHA', 'CHO').replace('BKN', 'BRK').replace('PHX', 'PHO')

In [200]:
teams.to_csv('teams_cleaned.csv', index = False)

# Getting NBA roster data

In [199]:
team_abbreviations = teams['bbreference_abbreviation'].to_list()

In [ ]:
response.encoding = 'utf-8'

In [191]:
# Function to extract player names, player IDs, and jersey numbers
def get_nba_roster(team_abbr):
    # Construct the URL for the team roster for the 2025 season
    url = f'https://www.basketball-reference.com/teams/{team_abbr}/2025.html'
    
    # Send a GET request to the webpage
    response = requests.get(url)
    
    # If the request was successful
    if response.status_code == 200:

        response.encoding = 'utf-8'
        
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find the table with player rosters (the table has a class 'sortable stats_table')
        table = soup.find('table', {'class': 'sortable stats_table'})
        
        # List to store players, their IDs, and jersey numbers
        players = []
        
        # Iterate through each row of the table (excluding the header)
        for row in table.find_all('tr')[1:]:
            # Find the player's jersey number (in the <th> with the 'number' stat)
            number_cell = row.find('th', {'data-stat': 'number'})
            player_number = number_cell.text.strip() if number_cell else 'N/A'
            
            # Find the player name and player ID (in the <a> tag)
            player_cell = row.find('td', {'data-stat': 'player'})
            if player_cell:
                player_link = player_cell.find('a')
                if player_link:
                    #player_name = player_link.text
                    player_id = player_link['href'].split('/')[3].split('.')[0]  # Extract player ID from the href
                    
                    players.append((player_id, team_abbr, player_number))
        
        return players
    else:
        print(f"Failed to retrieve data for {team_abbr} (Status code: {response.status_code})")
        return []

In [205]:
roster_dfs = []
for team in team_abbreviations:
    players = get_nba_roster(team)
    print(f'Received roster info for: {team}')
    if players:
        df = pd.DataFrame(players, columns = ['player_id','bbreference_abbreviation', 'player_number'])
    roster_dfs.append(df)
    time.sleep(5)

Received roster info for: ATL
Received roster info for: BOS
Received roster info for: BRK
Received roster info for: CHO
Received roster info for: CHI
Received roster info for: CLE
Received roster info for: DAL
Received roster info for: DEN
Received roster info for: DET
Received roster info for: GSW
Received roster info for: HOU
Received roster info for: IND
Received roster info for: LAC
Received roster info for: LAL
Received roster info for: MEM
Received roster info for: MIA
Received roster info for: MIL
Received roster info for: MIN
Received roster info for: NOP
Received roster info for: NYK
Received roster info for: OKC
Received roster info for: ORL
Received roster info for: PHI
Received roster info for: PHO
Received roster info for: POR
Received roster info for: SAC
Received roster info for: SAS
Received roster info for: TOR
Received roster info for: UTA
Received roster info for: WAS


In [206]:
test = pd.concat(roster_dfs, ignore_index = True)

In [207]:
test.head(100)

,player_id,bbreference_abbreviation,player_number
0,youngtr01,ATL,11
1,daniedy01,ATL,5
2,okongon01,ATL,17
3,risacza01,ATL,10
4,capelca01,ATL,15
5,mathega01,ATL,24
6,krejcvi01,ATL,27
7,johnsja05,ATL,1
8,wallake01,ATL,2
9,nancela02,ATL,22
